# Initializing Dash for visualization

## imports

In [1]:
import os
import sys
import pandas as pd
from PIL import Image
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import csv 

## get all the teams

In [5]:
sys.path.insert(0,'../src/data/')
from team_data import  get_all_teams 

seasons = [2016, 2017, 2018, 2019, 2020]    

fname = "../data/NHL_Teams_List.csv"
if os.path.isfile(fname):
    teams_lst_df=pd.read_csv(fname)
    teams_lst = teams_lst_df['Teams'].values.tolist()
else:
    teams_lst = get_all_teams(data_dir="../data/raw")
    teams_lst_df = pd.DataFrame(teams_lst,columns =['Teams'])
    teams_lst_df.to_csv(fname)
    



## Adding the graph, slider and two dropdown lists 
for the NHL rink, year, team and season respectively

In [6]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    html.Div([
        "Team: ",
        dcc.Dropdown(
                    id='teams-dropdown',
                    options=[{'label': i, 'value': i} for i in teams_lst],
                    value=teams_lst[0]
                ),
        "Season: ",
        dcc.Dropdown(
                    id='seasons-dropdown',
                    options=[{'label': i, 'value': str(i)} for i in seasons],
                    value=seasons[0]
                ),
        "Sub-season: ",
        dcc.Dropdown(
                    id='sub-seasons-dropdown',
                    options=[{'label': i, 'value': i} for i in ['regular', 'postseason']],
                    value='regular'
                ),
        ],
        style={'width': '49%', 'display': 'inline-block'}),
    
    html.Div([
        dcc.Graph(id='nhl-rink-graph'),
    ])
])

## Visualizing the shots for a given team, year and season

In [7]:
game_data_dir = "../data/games/"

In [8]:
def flip_coord_to_one_side(team_df):
    for idx, event_row in team_df.iterrows():
        if event_row['coordinate_x'] < 0:
            team_df.at[idx,'coordinate_x']= event_row['coordinate_x'] * -1
            team_df.at[idx,'coordinate_y']= event_row['coordinate_y'] * -1

In [9]:
@app.callback(
    Output('nhl-rink-graph', 'figure'),
    [Input('seasons-dropdown', 'value'),
     Input('teams-dropdown', 'value'),
     Input('sub-seasons-dropdown', 'value')])
def update_figure(selected_season, selected_team, selected_sub_season):
    img = Image.open('../figures/nhl_rink.png')
    
    # get the dataframe of the chosen team
    if selected_sub_season == 'regular':
        sub_str = '02'
    elif selected_sub_season == 'postseason':
        sub_str = '03'

    game_file_str = str(selected_season) + '-' + selected_sub_season + '.csv'
    game_df = pd.read_csv(os.path.join(game_data_dir, game_file_str))
    
    # team's data
    team_df = game_df[game_df['shooter_team_name'] == selected_team]
    team_df = team_df[team_df['home_team'] == selected_team]
    # for debugging
    # team_df = team_df[team_df['period'] == selected_period]

    
    fig = px.scatter(team_df, x="coordinate_x", y="coordinate_y", color="shooter_name")

    
    fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,)),
                  selector=dict(mode='markers'))
    
    # set the axes
    fig.update_yaxes(range=[-45, 45], 
                     scaleratio = 1, 
                     scaleanchor="y",
                     showgrid=True,
                     zeroline=True,
                     showline=True,
                     gridcolor='#bdbdbd',
                     gridwidth=2,
                     zerolinecolor='#969696',
                     zerolinewidth=4,
                     linecolor='#636363',
                     linewidth=6)
    fig.update_xaxes(range=[-100, 100], 
                     scaleratio = 1, 
                     scaleanchor="x",
                     showgrid=True,
                     zeroline=True,
                     showline=True,
                     gridcolor='#bdbdbd',
                     gridwidth=2,
                     zerolinecolor='#969696',
                     zerolinewidth=4,
                     linecolor='#636363',
                     linewidth=6)

    # remove any margins and padding
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        margin=dict(l=0, #left margin
                    r=0, #right margin
                    b=0, #bottom margin
                    t=0,  #top margin
                    pad=0
                   )
    )


    # Add the NHL rink image
    fig.add_layout_image(
            dict(
                source=img,
                xref="x",
                yref="y",
                x=0,
                y=0,
                sizex=200,
                sizey=85,
                sizing= "stretch",
                layer = "below",
                xanchor = "center",
                yanchor = "middle",
                opacity = 0.8),
    )

    return fig    

app.run_server()

Dash app running on http://127.0.0.1:8050/
